In [ ]:
import os
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.applications import MobileNet, VGG16
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [ ]:
for idx, class_name in enumerate(sentiment_folders):
        class_path = os.path.join(data_path, class_name)
        print(f"Class path: {class_path}")
        for image_file in os.listdir(class_path):
            image_path = os.path.join(class_path, image_file)
            print(f"Image path: {image_path}")
            try:
                image = cv2.imread(image_path)
                if image is None:
                    print(f"Error: Unable to read image at {image_path}")
                    continue

                # Resize the image to 48x48 pixels
                image = cv2.resize(image, (48, 48))
                # Normalize the image
                image = image / 255.0

                # Append the image and corresponding label
                x_data.append(image)
                y_data.append(idx)  # Label index based on folder position
                except cv2.error as e:
                print(f"Error: Unable to read image at {imagepath} due to formatting issues: {e}")

In [ ]:
def load_dataset(data_path, image_size=(48, 48)):
    x_data = []
    y_data = []
    # List of existing alphabet folders
    alphabet_folders = [ch for ch in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ' if ch not in ['H', 'J', 'Y']]

    for idx, class_name in enumerate(alphabet_folders):
        class_path = os.path.join(data_path, class_name)
        for image_file in os.listdir(class_path):
            image_path = os.path.join(class_path, image_file)
            image = cv2.imread(image_path)

            # Resize the image to 126x126 pixels
            image = cv2.resize(image, image_size)
            # Normalize the image
            image = image / 255.0

            # Append the image and corresponding label
            x_data.append(image)
            y_data.append(idx)  # Label index based on folder position

In [ ]:
# Convert lists to numpy arrays
x_data = np.array(x_data)
y_data = np.array(y_data)

    # Convert labels to one-hot encoding
y_data = to_categorical(y_data, num_classes=len(alphabet_folders))

# Usage:
data_path = '/Users/divyash/Downloads/isl/ISL_Dataset'  # Update with the path to your dataset
x_data, y_data, alphabet_folders = load_dataset(data_path)
test_size = 0.1
validation_size = 0.1
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=test_size, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=validation_size, random_state=42)
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
output = Dense(23, activation='softmax')(x)  # 26 classes excluding H, J, Y

model_sll = Model(inputs=base_model.input, outputs=output)
# Compile the sign language recognition model
model_sll.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the sign language recognition model
model_sll.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_val, y_val))

In [ ]:
# Evaluate the sign language recognition model on the test set
test_loss, test_accuracy = model_sll.evaluate(x_test, y_test)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")
Initialize MediaPipe models
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
mp_face_mesh = mp.solutions.face_mesh

# Initialize the video capture object
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Initialize the hand and face models
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1)
face_model = mp_face_mesh.FaceMesh(static_image_mode=False)
face_results = face_model.process(frame_rgb)
results = hands.process(frame_rgb)
    
if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Preprocess hand image for sign language recognition
            frame_resized = cv2.resize(frame, (126, 126))
            frame_normalized = frame_resized / 255.0
            frame_normalized = np.expand_dims(frame_normalized, axis=0)
# Extract face bounding box coordinates
            face_coords = []
            for landmark in face_landmarks.landmark:
                x = min(max(int(landmark.x * frame.shape[1]), 0), frame.shape[1] - 1)
                y = min(max(int(landmark.y * frame.shape[0]), 0), frame.shape[0] - 1)
                face_coords.append((x, y))

            # Calculate the bounding box of the face region
            min_x = min([x for x, y in face_coords])
            max_x = max([x for x, y in face_coords])
            min_y = min([y for x, y in face_coords])
            max_y = max([y for x, y in face_coords])

In [ ]:
# Extract the face region from the frame
face_region = frame[min_y:max_y, min_x:max_x]

            # Preprocess the face region for sentiment recognition
face_region_resized = cv2.resize(face_region, (48, 48))
face_region_normalized = face_region_resized / 255.0
ace_region_normalized = np.expand_dims(face_region_normalized, axis=0)
# Display the sentiment prediction on the frame
cv2.putText(frame, f"Sentiment: {predicted_sentiment_label}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

cv2.imshow('MediaPipe Hand and Face Landmarks', frame)

cap.release()
cv2.destroyAllWindows()